# Imports

In [689]:
import pandas as pd
import numpy as np
import itertools

# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# import joblib

# import psycopg2 as ps

# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
# from sklearn.pipeline import Pipeline

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

import os

In [690]:
PATH_IGNORE = 'ignore'
PATH_DATA = 'data'

FILE_DATA = os.path.join(PATH_DATA, 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx')
FILE_OLD_DATA = os.path.join(PATH_DATA, 'PEDE_PASSOS_DATASET_FIAP_ANTIGO.xlsx')
FILE_JOINED_DATA = os.path.join(PATH_DATA, 'data_joined.csv')
FILE_JOINED_DATA2022 = os.path.join(PATH_DATA, 'data_joined_2022.csv')
FILE_JOINED_DATA2023 = os.path.join(PATH_DATA, 'data_joined_2023.csv')
FILE_JOINED_DATA2024 = os.path.join(PATH_DATA, 'data_joined_2024.csv')

FILE_POSTGRES_PW = os.path.join(PATH_IGNORE, 'postgres-pw.txt')

In [691]:
raw_data = {}

raw_data[2022] = pd.read_excel(FILE_DATA, sheet_name=0)
raw_data[2023] = pd.read_excel(FILE_DATA, sheet_name=1)
raw_data[2024] = pd.read_excel(FILE_DATA, sheet_name=2)

# Dicionário

**INDE** - Índice de desenvolvimento educacional
- Ponderação entre os indicadores abaixo

**IAN** - Indicador de adequação de nível
- Registros administrativos
- Registra a defasagem da aprendizagem do aluno por meio da comparação da Fase atual do aluno da Associação, com a equivalência das Fases de ensido na Associação e a divisão dos anos escolares do ensino formal (por idade)
- Utiliza a fase atual, e a fase ideal conforme a idade.
- 10 (em fase), 5 (moderada) ou 2,5 (severa).

**IDA** - Indicador de desempenho acadêmico
- Notas Provas PM e Média geral Universitária
- Registra a proficiência do aluno nos exames padronizados de avaliação interna das disciplinas oferecidas pela Associação (os alunos das Fases 0 a 7). No caso dos alunos da Fase 8, esse indicador registra a nota média obtida pelos alunos em todas as disciplinas curriculares cursadas, nas respectivas instituições de ensino superior conveniadas ao programa de bolsas de estudo da Associação.
- Utiliza as notas internas da associação. Matemática + Português + Inglês / 3
- 0 a 10

**IEG** - Indicador de engajamento
- Registros de entrega de lição de casa e de voluntariado
- Mede o engajamento do aluno nas tarefas curriculares requeridas em cada uma das disciplinas oferecidas na Associação, e/ou seu engajamento em ações de voluntariado desenvolvidas pela Associação.
- Soma das pontuações das tarefas realizadas e registradas / Número de tarefas

**IAA** - Indicador de Autoavaliação
- Questionário de Autoavaliação individual
- Registra por meio de um questionário padronizado e adaptado às distintas faixas etárias dos alunoa a Associação, uma autoavaliação do aluno sobre como se sente consigo mesmo, sobre os estudos, sobre sua família, amigos e comunidade, e e sobre como se sente a respeito da Associação Passos Mágicos.
- Soma das pontuações das respostas do estudante / Número total de perguntas

**IPS** - Indicador Psicossocial
- Questionário individual de avaliação das psicólogas
- Avaliação da equipe de psicólogas para caracterizar o desenvolvimento do aluno nas suas interações familiares, no seu desenvolvimento emocional, comportamental e da sua socialização na vida comunitária. Esse indicador também caracteriza o tipo de atendimento psicológico oferecido pela Associação ao aluno.
- Avaliações feitas por **psicólogos**.
- Soma das pontuações / Número de avaliadores

**IPP** - Indicador Psicopedagógico
- Questionário individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos para caracterizar o desenvolvimento cognitivo, emocional, comportamental e de socialização do aluno no seu processo de aprendizado dentro do Programa de Aceleração do Conhecimento, dos Programas Educacionais e das Atividades Culturais promovidas pela Associação.
- Soma das avaliações sobre aspectos **pedagógicos** / Número de avaliações

**IPV** - Indicador de Ponto de Virada
- Questionário Individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos a respeito do desenvolvimento do aluno das aptidões necessárias para iniciar a transformação da sua vida por meio da Educação, avaliando a integração do aluno à Associação, o seu desenvolvimento emocional, e o seu potencial acadêmico.
- Análises longitudinais de progresso acadêmico, engajamento e desenvolvimento emocional

**Pedras** - Baseadas no INDE
- Quartzo: 3.0 < INDE < 6.1
- Ágata: 6.1 < INDE < 7.2
- Ametista: 7.2 < INDE < 8.2
- Topázio: 8.2 < INDE < 9.4

# Tratando dados
## Padronizando colunas
As colunas são inconsistente entre os datasets. Vamos extrair os dados que queremos um por um, ano por ano.

In [692]:
dfs_yearly = {}

dfs_yearly[2022] = pd.DataFrame()
dfs_yearly[2023] = pd.DataFrame()
dfs_yearly[2024] = pd.DataFrame()

In [693]:
def display_column_from_all(
    col : str,
    head : int = 3
):
    for df in dfs_yearly.values():
        display(df[col].head(head))

In [694]:
def display_value_counts_from_all(
	col : str
):
	for df in dfs_yearly.values():
		print(df[col].value_counts())

In [695]:
def replace_value_with(
	df,
	col,
	value,
	replace_with = np.nan
):
    to_na = df[col] == value # mask
    df.loc[to_na, col] = replace_with    

In [696]:
def extract_data(
    col_name : str,
    col_2022 : str,
    col_2023 : str,
    col_2024 : str
):
	# 2022
	year = 2022
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2022]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

	# 2023
	year = 2023
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2023]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

	# 2024
	year = 2024
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2024]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

In [697]:
def process_text(
	df,
	col
):
	try:
		df[col] = df[col].str.capitalize()
	except Exception as e:
		print(f'Failed to process text on df[{col}] | {e}')

In [698]:
def process_text_on_all_years(
        col
):
    for year in dfs_yearly:
        df = dfs_yearly[year]
        process_text(df, col)

### Alunos

In [699]:
col = 'aluno'

extract_data(
	col,
	'Nome',
	'Nome Anonimizado',
	'Nome Anonimizado'
)

In [700]:
display_column_from_all(col)

0    Aluno-1
1    Aluno-2
2    Aluno-3
Name: aluno, dtype: object

0    Aluno-861
1    Aluno-862
2    Aluno-863
Name: aluno, dtype: object

0    Aluno-1275
1    Aluno-1276
2    Aluno-1277
Name: aluno, dtype: object

### Idade

In [701]:
col = 'idade'

extract_data(
	col,
	'Idade 22',
	'Idade',
	'Idade'
)

In [702]:
display_column_from_all(col)
display_value_counts_from_all(col)

0    19
1    17
2    17
Name: idade, dtype: int64

0    8
1    9
2    7
Name: idade, dtype: object

0    8
1    8
2    8
Name: idade, dtype: int64

idade
10    116
12    107
11    102
13    100
9      96
14     85
15     63
8      57
16     52
17     48
7      14
18     12
19      5
20      2
21      1
Name: count, dtype: int64
idade
11                     77
10                     77
13                     65
9                      65
12                     62
1900-01-11 00:00:00    56
8                      55
14                     54
1900-01-12 00:00:00    47
1900-01-09 00:00:00    43
15                     43
1900-01-14 00:00:00    42
1900-01-10 00:00:00    36
16                     36
1900-01-08 00:00:00    34
1900-01-13 00:00:00    30
17                     25
1900-01-15 00:00:00    22
1900-01-17 00:00:00    21
1900-01-16 00:00:00    19
18                     18
1900-01-07 00:00:00    14
7                      13
1900-01-18 00:00:00    11
1900-01-19 00:00:00    10
1900-01-20 00:00:00    10
19                      8
21                      6
20                      5
22                      4
1900-01-21 00:00:00     3
1900-0

In [703]:
is_int = pd.Series()


count = 0

for row in dfs_yearly[2023][col]:
    is_int[count] = str(type(row)) == "<class 'int'>"
    count += 1

is_int.index = dfs_yearly[2023].index

In [704]:
dfs_yearly[2023].loc[is_int == False, 'idade'] = pd.to_datetime(dfs_yearly[2023].loc[is_int == False, 'idade']).dt.day
dfs_yearly[2023].loc[is_int == False, 'idade']

3        8
5        7
7        7
12      11
15       8
        ..
1000    17
1005    18
1006    21
1011    18
1013    18
Name: idade, Length: 399, dtype: object

In [705]:
dfs_yearly[2023]['idade'] = dfs_yearly[2023]['idade'].astype(int)

In [707]:
display_column_from_all(col)
display_value_counts_from_all(col)

0    19
1    17
2    17
Name: idade, dtype: int64

0    8
1    9
2    7
Name: idade, dtype: int64

0    8
1    8
2    8
Name: idade, dtype: int64

idade
10    116
12    107
11    102
13    100
9      96
14     85
15     63
8      57
16     52
17     48
7      14
18     12
19      5
20      2
21      1
Name: count, dtype: int64
idade
11    133
10    113
12    109
9     108
14     96
13     95
8      89
15     65
16     55
17     46
18     29
7      27
19     18
20     15
21      9
22      4
26      1
23      1
24      1
Name: count, dtype: int64
idade
12    148
11    132
10    128
13    103
9     102
15     98
14     97
16     80
8      68
17     59
18     26
21     25
7      20
19     20
20     20
22     17
23      9
24      2
27      1
25      1
Name: count, dtype: int64


### Ano

In [553]:
col = 'ano'

year = 2022
dfs_yearly[year][col] = year

year = 2023
dfs_yearly[year][col] = year

year = 2024
dfs_yearly[year][col] = year

### Ano de nascimento

In [554]:
col = 'ano_nascimento'

extract_data(
	col,
	col_2022 = 'Ano nasc',
	col_2023 = 'Data de Nasc',
	col_2024 = 'Data de Nasc'
)

In [555]:
display_column_from_all(col)

0    2003
1    2005
2    2005
Name: ano_nascimento, dtype: int64

0    6/17/2015
1    5/31/2014
2    2/25/2016
Name: ano_nascimento, dtype: object

0   2016-07-28
1   2016-10-16
2   2016-08-16
Name: ano_nascimento, dtype: datetime64[ns]

In [556]:
dfs_yearly[2022][col] = dfs_yearly[2022][col].astype('int32')

In [557]:
dfs_yearly[2023][col] = pd.to_datetime(dfs_yearly[2023][col])
dfs_yearly[2023][col] = dfs_yearly[2023][col].dt.year

In [558]:
dfs_yearly[2024][col] = dfs_yearly[2024][col].dt.year

In [559]:
display_column_from_all(col)

0    2003
1    2005
2    2005
Name: ano_nascimento, dtype: int32

0    2015
1    2014
2    2016
Name: ano_nascimento, dtype: int32

0    2016
1    2016
2    2016
Name: ano_nascimento, dtype: int32

### INDE

In [560]:
col = 'inde'

extract_data(
    col,
    col_2022 = 'INDE 22',
    col_2023 = 'INDE 2023',
    col_2024 = 'INDE 2024'
)

display_column_from_all('inde')

0    5.783
1    7.055
2    6.591
Name: inde, dtype: float64

0    9.31095
1    8.22120
2    5.92975
Name: inde, dtype: float64

0    7.611367
1    8.002867
2      7.9522
Name: inde, dtype: object

In [561]:
dfs_yearly[2024][col].describe()

count        1092
unique       1055
top       INCLUIR
freq           38
Name: inde, dtype: object

In [562]:
dfs_yearly[2024][col].value_counts()

inde
INCLUIR     38
8.002867     1
7.9522       1
7.156367     1
5.4442       1
            ..
4.78185      1
7.785233     1
8.27555      1
8.0853       1
7.611367     1
Name: count, Length: 1055, dtype: int64

In [563]:
replace_value_with(
    dfs_yearly[2024],
    col,
    'INCLUIR',
    np.nan
)

In [564]:
dfs_yearly[2024][col] = dfs_yearly[2024][col].astype(float)

In [565]:
display_column_from_all(col)

0    5.783
1    7.055
2    6.591
Name: inde, dtype: float64

0    9.31095
1    8.22120
2    5.92975
Name: inde, dtype: float64

0    7.611367
1    8.002867
2    7.952200
Name: inde, dtype: float64

### Pedra 22

In [566]:
col = 'pedra_22'

extract_data(
	col,
	col_2022 = 'Pedra 22',
	col_2023 = 'Pedra 22',
	col_2024 = 'Pedra 22'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0     Quartzo
1    Ametista
2       Ágata
Name: pedra_22, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_22, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_22, dtype: object

pedra_22
Ametista    348
Ágata       250
Quartzo     132
Topázio     130
Name: count, dtype: int64
pedra_22
Ametista    264
Ágata       165
Topázio     118
Quartzo      53
Name: count, dtype: int64
pedra_22
Ametista    218
Ágata       117
Topázio     106
Quartzo      31
Name: count, dtype: int64


### Pedra 23

In [567]:
col = 'pedra_23'

extract_data(
	col,
	col_2022 = 'Pedra 23',
	col_2023 = 'Pedra 2023',
	col_2024 = 'Pedra 23'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

Failed to find column on 2022 | 'Pedra 23'
Failed to process text on df[pedra_23] | Can only use .str accessor with string values!


0   NaN
1   NaN
2   NaN
Name: pedra_23, dtype: float64

0    Topázio
1    Topázio
2    Quartzo
Name: pedra_23, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_23, dtype: object

Series([], Name: count, dtype: int64)
pedra_23
Ametista    381
Agata       246
Topázio     232
Quartzo      72
Name: count, dtype: int64
pedra_23
Ametista    308
Topázio     187
Agata       153
Quartzo      42
Name: count, dtype: int64


### Pedra 24

In [568]:
col = 'pedra_24'

extract_data(
	col,
	col_2022 = 'Pedra 24',
	col_2023 = 'Pedra 24',
	col_2024 = 'Pedra 2024'
)

display_column_from_all(col)
display_value_counts_from_all(col)

Failed to find column on 2022 | 'Pedra 24'
Failed to find column on 2023 | 'Pedra 24'


0   NaN
1   NaN
2   NaN
Name: pedra_24, dtype: float64

0   NaN
1   NaN
2   NaN
Name: pedra_24, dtype: float64

0    Ametista
1     Topázio
2    Ametista
Name: pedra_24, dtype: object

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
pedra_24
Ametista    391
Topázio     326
Agata       225
Quartzo     112
INCLUIR      38
Name: count, dtype: int64


In [569]:
replace_value_with(
    dfs_yearly[2024],
    col,
    'INCLUIR'
)

In [570]:
display_value_counts_from_all(
    col
)

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
pedra_24
Ametista    391
Topázio     326
Agata       225
Quartzo     112
Name: count, dtype: int64


### Gênero

In [571]:
col = 'genero'

extract_data(
	col,
	col_2022 = 'Gênero',
	col_2023 = 'Gênero',
	col_2024 = 'Gênero'
)

display_column_from_all(col)
display_value_counts_from_all(col)

0    Menina
1    Menina
2    Menina
Name: genero, dtype: object

0     Feminino
1    Masculino
2    Masculino
Name: genero, dtype: object

0    Masculino
1     Feminino
2    Masculino
Name: genero, dtype: object

genero
Menina    457
Menino    403
Name: count, dtype: int64
genero
Feminino     546
Masculino    468
Name: count, dtype: int64
genero
Feminino     623
Masculino    533
Name: count, dtype: int64


In [572]:
dfs_yearly[2022] = dfs_yearly[2022].replace(
    {
        'Menina':'Feminino',
        'Menino':'Masculino'
	}
)

In [573]:
process_text_on_all_years(col)
display_column_from_all(col)

0    Feminino
1    Feminino
2    Feminino
Name: genero, dtype: object

0     Feminino
1    Masculino
2    Masculino
Name: genero, dtype: object

0    Masculino
1     Feminino
2    Masculino
Name: genero, dtype: object

### Ano de ingresso

In [574]:
col = 'ano_ingresso'

extract_data(
	col,
	col_2022 = 'Ano ingresso',
	col_2023 = 'Ano ingresso',
	col_2024 = 'Ano ingresso'
)

display_column_from_all(col)
display_value_counts_from_all(col)

0    2016
1    2017
2    2016
Name: ano_ingresso, dtype: int64

0    2023
1    2023
2    2023
Name: ano_ingresso, dtype: int64

0    2024
1    2024
2    2024
Name: ano_ingresso, dtype: int64

ano_ingresso
2022    399
2019    139
2021    138
2018     67
2020     48
2017     40
2016     29
Name: count, dtype: int64
ano_ingresso
2023    369
2022    268
2021    137
2019     98
2018     46
2020     40
2017     32
2016     24
Name: count, dtype: int64
ano_ingresso
2024    349
2021    345
2023    248
2022    214
Name: count, dtype: int64


### Instituição de ensino

In [575]:
col = 'instituicao_ensino'

extract_data(
	col,
	col_2022 = 'Instituição de ensino',
	col_2023 = 'Instituição de ensino',
	col_2024 = 'Instituição de ensino'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0    Escola pública
1      Rede decisão
2      Rede decisão
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

instituicao_ensino
Escola pública    752
Rede decisão      106
Escola jp ii        2
Name: count, dtype: int64
instituicao_ensino
Pública                                      809
Privada - programa de apadrinhamento         100
Privada *parcerias com bolsa 100%             60
Privada                                       28
Privada - pagamento por *empresa parceira      8
Concluiu o 3º em                               7
Nenhuma das opções acima                       2
Name: count, dtype: int64
instituicao_ensino
Pública                                      913
Privada - programa de apadrinhamento          96
Privada                                       76
Privada *parcerias com bolsa 100%             41
Bolsista universitário *formado (a)           13
Privada - pagamento por *empresa parceira      9
Concluiu o 3º em                               7
Name: count, dtype: int64


In [576]:
dfs_yearly[2022] = dfs_yearly[2022].replace(
	{
        'Escola pública' : 'Pública',
	}
)

In [577]:
process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0         Pública
1    Rede decisão
2    Rede decisão
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

instituicao_ensino
Pública         752
Rede decisão    106
Escola jp ii      2
Name: count, dtype: int64
instituicao_ensino
Pública                                      809
Privada - programa de apadrinhamento         100
Privada *parcerias com bolsa 100%             60
Privada                                       28
Privada - pagamento por *empresa parceira      8
Concluiu o 3º em                               7
Nenhuma das opções acima                       2
Name: count, dtype: int64
instituicao_ensino
Pública                                      913
Privada - programa de apadrinhamento          96
Privada                                       76
Privada *parcerias com bolsa 100%             41
Bolsista universitário *formado (a)           13
Privada - pagamento por *empresa parceira      9
Concluiu o 3º em                               7
Name: count, dtype: int64


### IAN

In [578]:
col = 'ian'

extract_data(
	col,
	col_2022 = 'IAN',
	col_2023 = 'IAN',
	col_2024 = 'IAN'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0     5.0
1    10.0
2    10.0
Name: ian, dtype: float64

0    10.0
1     5.0
2    10.0
Name: ian, dtype: float64

0    10.0
1    10.0
2    10.0
Name: ian, dtype: float64

### IDA

In [579]:
col = 'ida'

extract_data(
	col,
	col_2022 = 'IDA',
	col_2023 = 'IDA',
	col_2024 = 'IDA'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    4.0
1    6.8
2    5.6
Name: ida, dtype: float64

0    9.6
1    8.9
2    6.3
Name: ida, dtype: float64

0    8.0
1    8.0
2    8.0
Name: ida, dtype: float64

### IEG

In [580]:
col = 'ieg'

extract_data(
	col,
	col_2022 = 'IEG',
	col_2023 = 'IEG',
	col_2024 = 'IEG'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    4.1
1    5.2
2    7.9
Name: ieg, dtype: float64

0    10.0
1     9.1
2     7.6
Name: ieg, dtype: float64

0    8.666667
1    9.333333
2    9.083333
Name: ieg, dtype: float64

### IAA

In [581]:
col = 'iaa'

extract_data(
	col,
	col_2022 = 'IAA',
	col_2023 = 'IAA',
	col_2024 = 'IAA'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    8.3
1    8.8
2    0.0
Name: iaa, dtype: float64

0    9.5
1    8.5
2    0.0
Name: iaa, dtype: float64

0    10.002
1    10.002
2    10.002
Name: iaa, dtype: float64

### IPS

In [582]:
col = 'ips'

extract_data(
	col,
	col_2022 = 'IPS',
	col_2023 = 'IPS',
	col_2024 = 'IPS'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    5.6
1    6.3
2    5.6
Name: ips, dtype: float64

0    8.13
1    8.14
2    3.14
Name: ips, dtype: float64

0    6.26
1    3.76
2    3.76
Name: ips, dtype: float64

### IPP

In [583]:
col = 'ipp'

extract_data(
	col,
	col_2022 = 'IPP',
	col_2023 = 'IPP',
	col_2024 = 'IPP'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

Failed to find column on 2022 | 'IPP'


0   NaN
1   NaN
2   NaN
Name: ipp, dtype: float64

0    8.4375
1    7.5000
2    5.9375
Name: ipp, dtype: float64

0    5.625
1    7.500
2    7.500
Name: ipp, dtype: float64

### IPV

In [584]:
col = 'ipv'

extract_data(
	col,
	col_2022 = 'IPV',
	col_2023 = 'IPV',
	col_2024 = 'IPV'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    7.278
1    6.778
2    7.556
Name: ipv, dtype: float64

0    8.920
1    8.585
2    6.260
Name: ipv, dtype: float64

0    5.446667
1    7.050000
2    7.046667
Name: ipv, dtype: float64

### Matemática

In [585]:
col = 'nota_mat'

extract_data(
	col,
	col_2022 = 'Matem',
	col_2023 = 'Mat',
	col_2024 = 'Mat'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    2.7
1    6.3
2    5.8
Name: nota_mat, dtype: float64

0    9.8
1    8.5
2    7.0
Name: nota_mat, dtype: float64

0    10.0
1    10.0
2    10.0
Name: nota_mat, dtype: float64

### Português

In [586]:
col = 'nota_port'

extract_data(
	col,
	col_2022 = 'Portug',
	col_2023 = 'Por',
	col_2024 = 'Por'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    3.5
1    4.5
2    4.0
Name: nota_port, dtype: float64

0    9.4
1    9.2
2    5.5
Name: nota_port, dtype: float64

0    6.0
1    6.0
2    6.0
Name: nota_port, dtype: float64

### Inglês

In [587]:
col = 'nota_ing'

extract_data(
	col,
	col_2022 = 'Inglês',
	col_2023 = 'Ing',
	col_2024 = 'Ing'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    6.0
1    9.7
2    6.9
Name: nota_ing, dtype: float64

0   NaN
1   NaN
2   NaN
Name: nota_ing, dtype: float64

0   NaN
1   NaN
2   NaN
Name: nota_ing, dtype: float64

## Mergendo dados

In [588]:
for year in dfs_yearly:
    dfs_yearly[year] = dfs_yearly[year].set_index('aluno')
dfs_yearly[2022]

,idade,ano,ano_nascimento,inde,pedra_22,pedra_23,pedra_24,genero,ano_ingresso,instituicao_ensino,ian,ida,ieg,iaa,ips,ipp,ipv,nota_mat,nota_port,nota_ing
aluno,,,,,,,,,,,,,,,,,,,,
Aluno-1,19,2022,2003,5.783,Quartzo,NaN,NaN,Feminino,2016,Pública,5.0,4.0,4.1,8.3,5.6,NaN,7.278,2.7,3.5,6.0
Aluno-2,17,2022,2005,7.055,Ametista,NaN,NaN,Feminino,2017,Rede decisão,10.0,6.8,5.2,8.8,6.3,NaN,6.778,6.3,4.5,9.7
Aluno-3,17,2022,2005,6.591,Ágata,NaN,NaN,Feminino,2016,Rede decisão,10.0,5.6,7.9,0.0,5.6,NaN,7.556,5.8,4.0,6.9
Aluno-4,17,2022,2005,5.951,Quartzo,NaN,NaN,Masculino,2017,Rede decisão,10.0,5.0,4.5,8.8,5.6,NaN,5.278,2.8,3.5,8.7
Aluno-5,17,2022,2005,7.427,Ametista,NaN,NaN,Feminino,2016,Rede decisão,10.0,5.2,8.6,7.9,5.6,NaN,7.389,7.0,2.9,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aluno-856,7,2022,2015,8.398,Topázio,NaN,NaN,Masculino,2022,Pública,10.0,8.4,9.1,10.0,7.5,NaN,8.000,8.5,8.2,NaN
Aluno-857,9,2022,2013,8.154,Topázio,NaN,NaN,Feminino,2022,Pública,5.0,9.5,8.9,8.5,7.5,NaN,9.083,9.6,9.3,NaN
Aluno-858,7,2022,2015,7.523,Ametista,NaN,NaN,Masculino,2022,Pública,10.0,7.7,7.1,10.0,5.6,NaN,7.333,8.4,6.9,NaN


### Lidando com dados anuais

In [589]:
data : dict[int, pd.DataFrame] = {}
data[2022] = dfs_yearly[2022].copy()
data[2023] = dfs_yearly[2023].copy()
data[2024] = dfs_yearly[2024].copy()

for year in data:
    data[year].columns = [f'{col}-{year}' for col in data[year].columns]

data[2022].head(3)

,idade-2022,ano-2022,ano_nascimento-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,genero-2022,ano_ingresso-2022,instituicao_ensino-2022,ian-2022,ida-2022,ieg-2022,iaa-2022,ips-2022,ipp-2022,ipv-2022,nota_mat-2022,nota_port-2022,nota_ing-2022
aluno,,,,,,,,,,,,,,,,,,,,
Aluno-1,19,2022,2003,5.783,Quartzo,NaN,NaN,Feminino,2016,Pública,5.0,4.0,4.1,8.3,5.6,NaN,7.278,2.7,3.5,6.0
Aluno-2,17,2022,2005,7.055,Ametista,NaN,NaN,Feminino,2017,Rede decisão,10.0,6.8,5.2,8.8,6.3,NaN,6.778,6.3,4.5,9.7
Aluno-3,17,2022,2005,6.591,Ágata,NaN,NaN,Feminino,2016,Rede decisão,10.0,5.6,7.9,0.0,5.6,NaN,7.556,5.8,4.0,6.9


In [590]:
students : dict[int, set] = {}

for year in data:
    students[year] = set(data[year].index)

In [591]:
# Alunos que estão presentes apenas nos registros de 2023 para frente
students[2023] = students[2023] - students[2022]
print(len(students[2023]))

# Alunos que estão presentes apenas nos registros de 2024 para frente
students[2024] = students[2024] - students[2022] - students[2023]
print(len(students[2024]))

414
387


#### Atualizando alunos de 2022 até 2024

In [592]:
df_students_2022 : pd.DataFrame

df_students_2022 = pd.merge(
	left = data[2022],
	right = data[2023],
	on = 'aluno',
	how = 'left'
)

df_students_2022 = pd.merge(
	left = df_students_2022,
	right = data[2024],
	on = 'aluno',
	how = 'left'
)

df_students_2022.head(3)

,idade-2022,ano-2022,ano_nascimento-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,genero-2022,ano_ingresso-2022,instituicao_ensino-2022,...,ian-2024,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-1,19,2022,2003,5.783,Quartzo,NaN,NaN,Feminino,2016,Pública,...,10.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-2,17,2022,2005,7.055,Ametista,NaN,NaN,Feminino,2017,Rede decisão,...,10.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-3,17,2022,2005,6.591,Ágata,NaN,NaN,Feminino,2016,Rede decisão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [593]:
df_students_2022.shape, len(students[2022])

((860, 60), 860)

#### Atualizando alunos de 2023 até 2024

In [594]:
df_students_2023 : pd.DataFrame

df_students_2023 = pd.merge(
	left = data[2023].loc[list(students[2023])],
	right = data[2024],
	on = 'aluno',
	how = 'left'
)

df_students_2023.head(3)

,idade-2023,ano-2023,ano_nascimento-2023,inde-2023,pedra_22-2023,pedra_23-2023,pedra_24-2023,genero-2023,ano_ingresso-2023,instituicao_ensino-2023,...,ian-2024,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-1212,16,2023,2007,6.755358,NaN,Agata,NaN,Feminino,2023,Pública,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-1123,1900-01-12 00:00:00,2023,2010,6.956500,NaN,Agata,NaN,Masculino,2023,Pública,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-1075,12,2023,2010,6.591333,NaN,Agata,NaN,Feminino,2023,Pública,...,5.0,6.0,8.515406,7.5,3.755,7.708333,7.553333,5.0,7.0,NaN


In [595]:
df_students_2023.shape, len(students[2023])

((414, 40), 414)

#### Pegando os alunos de 2024

In [596]:
df_students_2024 : pd.DataFrame

df_students_2024 = data[2024].loc[list(students[2024])]

df_students_2024.head(3)

,idade-2024,ano-2024,ano_nascimento-2024,inde-2024,pedra_22-2024,pedra_23-2024,pedra_24-2024,genero-2024,ano_ingresso-2024,instituicao_ensino-2024,ian-2024,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024
aluno,,,,,,,,,,,,,,,,,,,,
Aluno-1301,8,2024,2015,5.634533,NaN,NaN,Quartzo,Masculino,2024,Pública,5.0,7.0,2.916667,9.502,2.51,7.5000,6.000000,10.0,4.0,NaN
Aluno-1278,9,2024,2015,5.444200,NaN,NaN,Quartzo,Masculino,2024,Pública,5.0,7.5,4.166667,9.002,3.76,5.0000,4.173333,8.0,7.0,NaN
Aluno-1284,8,2024,2016,8.380015,NaN,NaN,Topázio,Feminino,2024,Pública,10.0,10.0,9.347826,9.002,2.51,7.8125,7.890000,10.0,10.0,NaN


In [597]:
df_students_2024.shape, len(students[2024])

((387, 20), 387)

### Juntando as sheets

#### Padronizando as colunas

In [598]:
cols = df_students_2022.columns

for df in [df_students_2023, df_students_2024]:
	for col in cols:
		if col not in df.columns:
			df[col] = np.nan

#### Concatenando

In [599]:
df = pd.concat(
    [df_students_2022,
    df_students_2023]
)

df = pd.concat(
    [df,
     df_students_2024]
)

df.shape

(1661, 60)

In [600]:
len(students[2022] | students[2023] | students[2024])

1661

In [601]:
df.dtypes

idade-2022                 float64
ano-2022                   float64
ano_nascimento-2022        float64
inde-2022                  float64
pedra_22-2022               object
pedra_23-2022              float64
pedra_24-2022              float64
genero-2022                 object
ano_ingresso-2022          float64
instituicao_ensino-2022     object
ian-2022                   float64
ida-2022                   float64
ieg-2022                   float64
iaa-2022                   float64
ips-2022                   float64
ipp-2022                   float64
ipv-2022                   float64
nota_mat-2022              float64
nota_port-2022             float64
nota_ing-2022              float64
idade-2023                  object
ano-2023                   float64
ano_nascimento-2023        float64
inde-2023                  float64
pedra_22-2023               object
pedra_23-2023               object
pedra_24-2023              float64
genero-2023                 object
ano_ingresso-2023   

## Checando união
Vamos ver se os alunos estão coerentes entre anos

In [602]:
def get_column_across_years(col):
    return [f'{col}-{year}' for year in dfs_yearly]

In [603]:
def columns_are_equal(df : pd.DataFrame, col_list : list):
	combinations : list[tuple] = [x for x in itertools.combinations(list(range(len(col_list))), 2)]
	# print(combinations)

	values = df[col_list]

	are_equal = pd.Series()
	count = 0

	for row in values.values:
		values_are_equal = True
		for combination in combinations:
			x = combination[0]
			y = combination[1]
			if row[x] == row[y]:
				pass
			else:
				values_are_equal = False
		
		are_equal[count] = values_are_equal
		count += 1


	are_equal.index = df.index
	return are_equal.all(), are_equal

#### Pedras

In [604]:
cols = get_column_across_years('pedra_22')
columns_are_equal(df, cols)[0]

np.False_

In [605]:
cols = ['pedra_23-2023', 'pedra_23-2024']
columns_are_equal(df, cols)[0]

np.False_

#### Ano de ingresso

In [606]:
cols = get_column_across_years('ano_ingresso')

df_check = df[cols].dropna()

check, are_equal = columns_are_equal(df_check, cols)
check

np.False_

In [607]:
df_check.loc[are_equal == False, cols].head(10)

,ano_ingresso-2022,ano_ingresso-2023,ano_ingresso-2024
aluno,,,
Aluno-1,2016.0,2016.0,2021.0
Aluno-2,2017.0,2017.0,2021.0
Aluno-7,2017.0,2017.0,2021.0
Aluno-12,2020.0,2020.0,2021.0
Aluno-13,2016.0,2016.0,2021.0
Aluno-27,2016.0,2016.0,2021.0
Aluno-31,2019.0,2019.0,2021.0
Aluno-35,2019.0,2019.0,2021.0
Aluno-36,2019.0,2019.0,2021.0


#### Ano de nascimento

In [608]:
cols = get_column_across_years('ano_nascimento')

df_check = df[cols].dropna()

check, are_equal = columns_are_equal(df_check, cols)
check

np.False_

In [609]:
df_check.loc[are_equal == False, cols].head(10)

,ano_nascimento-2022,ano_nascimento-2023,ano_nascimento-2024
aluno,,,
Aluno-27,2005.0,2006.0,2006.0
Aluno-158,2005.0,2009.0,2009.0
Aluno-161,2011.0,2009.0,2009.0
Aluno-178,2009.0,2009.0,2008.0
Aluno-184,2006.0,2010.0,2010.0
Aluno-444,2011.0,2011.0,2009.0
Aluno-454,2008.0,2008.0,2007.0


Vamos checar se as idades estão mais coerentes

In [610]:
df[get_column_across_years('idade')]

,idade-2022,idade-2023,idade-2024
aluno,,,
Aluno-1,19.0,1900-01-20 00:00:00,21.0
Aluno-2,17.0,18,19.0
Aluno-3,17.0,NaN,NaN
Aluno-4,17.0,NaN,NaN
Aluno-5,17.0,1900-01-18 00:00:00,NaN
...,...,...,...
Aluno-1633,NaN,NaN,21.0
Aluno-1587,NaN,NaN,16.0
Aluno-1285,NaN,NaN,7.0


## Criando dados novos

#### Desistência
Classificaremos como desistente o aluno que tinha dados no ano anterior, e não tem mais.

---

## Nulos

In [611]:
df = df.replace(
    'nan',
    np.nan
)

df = df.replace(
    'NA',
    np.nan
)

## Bases anuais

In [612]:
year = 2022
cols = fixed_columns[1:] + [ f'{col}_{year}' for col in yearly_columns ]
df_2022 = df[cols]

year = 2023
cols = fixed_columns[1:] + [ f'{col}_{year}' for col in yearly_columns ]
df_2023 = df[cols]

year = 2024
cols = fixed_columns[1:] + [ f'{col}_{year}' for col in yearly_columns ]
df_2024 = df[cols]

NameError: name 'fixed_columns' is not defined

# Tratando dados
Adicionando novos parâmetros, tratando dados existentes

In [ ]:
def drop_nulls(df):
    col = df.columns[-1:][0]
    to_drop = df[col].isna()
    to_drop = df[to_drop].index

    return df.drop(index = to_drop)

In [ ]:
df_2022 = drop_nulls(df_2022)
df_2023 = drop_nulls(df_2023)
df_2024 = drop_nulls(df_2024)

In [ ]:
_ = df_2024.instituicao_2024.isna()
_.value_counts()

instituicao_2024
False    1155
Name: count, dtype: int64

In [ ]:
df.isna().sum()

genero                 0
ano_ingresso           0
ano_nascimento         0
fase_2022            801
ian_2022             801
ida_2022             801
ieg_2022             801
iaa_2022             801
ips_2022             801
ipp_2022             801
ipv_2022             801
matematica_2022      803
portugues_2022       803
ingles_2022         1378
instituicao_2022     801
fase_2023            647
ian_2023             647
ida_2023             724
ieg_2023             723
iaa_2023             710
ips_2023             716
ipp_2023             723
ipv_2023             723
matematica_2023      724
portugues_2023       724
ingles_2023         1327
instituicao_2023     647
fase_2024            505
ian_2024             505
ida_2024             606
ieg_2024             505
iaa_2024             607
ips_2024             607
ipp_2024             607
ipv_2024             607
matematica_2024      610
portugues_2024       611
ingles_2024         1187
instituicao_2024     506
dtype: int64

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ano_ingresso,1661.0,2021.821192,1.976655,2016.000000,2021.000000,2022.000000,2023.000000,2024.000000
ano_nascimento,1661.0,2010.526791,3.478857,1996.000000,2008.000000,2011.000000,2013.000000,2017.000000
ian_2022,860.0,6.424419,2.389609,2.500000,5.000000,5.000000,10.000000,10.000000
ida_2022,860.0,6.092907,2.046209,0.000000,4.800000,6.300000,7.600000,9.900000
ieg_2022,860.0,7.891163,1.638340,0.000000,7.000000,8.300000,9.100000,10.000000
iaa_2022,860.0,8.274419,2.064935,0.000000,7.900000,8.800000,9.500000,10.000000
ips_2022,860.0,6.905000,1.070707,2.500000,6.300000,7.500000,7.500000,10.000000
ipp_2022,860.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ipv_2022,860.0,7.253642,1.093383,2.500000,6.722000,7.333000,7.917000,10.000000
matematica_2022,858.0,5.806876,2.414215,0.000000,4.300000,6.000000,7.800000,10.000000


### Desistências

In [ ]:
students_2022 = df_2022.index.to_list()
students_2023 = df_2023.index.to_list()
students_2024 = df_2024.index.to_list()

In [ ]:
terminated_2022 = []
# student from 2022 is in 2023?
for student in students_2022:
    if student not in students_2023:
        terminated_2022.append(student)


terminated_2023 = []
# student from 2023 is in 2024?
for student in students_2023:
    if student not in students_2024:
        terminated_2023.append(student)

print(
    len(terminated_2022),
    len(terminated_2023),
    )

260 249


In [ ]:
df['desistiu_no_ano_seguinte_2022'] = False
df_2022['desistiu_no_ano_seguinte_2022'] = False
df.loc[terminated_2022, 'desistiu_no_ano_seguinte_2022'] = True
df_2022.loc[terminated_2022, 'desistiu_no_ano_seguinte_2022'] = True


df['desistiu_no_ano_seguinte_2023'] = False
df_2023['desistiu_no_ano_seguinte_2023'] = False
df.loc[terminated_2023, 'desistiu_no_ano_seguinte_2023'] = True
df_2023.loc[terminated_2023, 'desistiu_no_ano_seguinte_2023'] = True

df_2023

,genero,ano_ingresso,ano_nascimento,fase_2023,ian_2023,ida_2023,ieg_2023,iaa_2023,ips_2023,ipp_2023,ipv_2023,matematica_2023,portugues_2023,ingles_2023,instituicao_2023,desistiu_no_ano_seguinte,desistiu_no_ano_seguinte_2023
nome_anonimizado,,,,,,,,,,,,,,,,,
Aluno-1,Menina,2016,2003,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
Aluno-2,Menina,2017,2005,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
Aluno-5,Menina,2016,2005,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,True
Aluno-6,Menina,2021,2004,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada - pagamento por *empresa parceira,False,False
Aluno-7,Menino,2017,2004,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aluno-1022,Masculino,2023,2014,fase 1,10.0,4.7,8.5,8.5,3.77,7.18750,7.893333,4.7,4.7,NaN,pública,False,False
Aluno-1231,Masculino,2021,2001,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
Aluno-948,Feminino,2023,2015,alfa,10.0,0.0,7.6,9.0,7.52,6.87500,8.340000,0.0,0.0,NaN,pública,False,True


## Médias por aluno

In [ ]:
df.head(2)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-1,Menina,2016,2003,7.0,5.0,4.0,4.1,8.3,5.6,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%
Aluno-2,Menina,2017,2005,7.0,10.0,6.8,5.2,8.8,6.3,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%


In [ ]:
df.ida_2022

nome_anonimizado
Aluno-1       4.0
Aluno-2       6.8
Aluno-3       5.6
Aluno-4       5.0
Aluno-5       5.2
             ... 
Aluno-1290    NaN
Aluno-1420    NaN
Aluno-1407    NaN
Aluno-1444    NaN
Aluno-1355    NaN
Name: ida_2022, Length: 1661, dtype: float64

## Export

In [ ]:
df.to_csv(FILE_JOINED_DATA)

df_2022.to_csv(FILE_JOINED_DATA2022)
df_2023.to_csv(FILE_JOINED_DATA2023)
df_2024.to_csv(FILE_JOINED_DATA2024)


# Conexão com o PostgreSQL

In [ ]:
def connect_to_db(
        dbname = 'datathon',
        user = 'postgres',
        password_path = FILE_POSTGRES_PW,
        host = 'localhost',
        port = '5432'
):
    password = ''
    with open(password_path) as f:
        password = f.read()

    conn = ps.connect(
        dbname = dbname,
        user = user,
        password = password,
        host = host,
        port = port
    )

    return conn

In [ ]:
cursor = None
conn = None
try:
    conn = connect_to_db()
    sql_create_schema = 'CREATE SCHEMA IF NOT EXISTS datathon;'
    
    cursor = conn.cursor()
    cursor.execute(sql_create_schema)
    conn.commit()

    print("Schema created successfully")
except Exception as e:
    
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Failed to create schema | connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



In [ ]:
sql_types = {
    'object' : 'VARCHAR(255)',
    'int64': 'INT',
    'float64': 'FLOAT'
}

schema_types = {}

for col in df.columns:
    schema_types[col] = sql_types[f'{df[col].dtype}']

schema_types

{'genero': 'VARCHAR(255)',
 'ano_ingresso': 'INT',
 'ano_nascimento': 'INT',
 'fase_2022': 'VARCHAR(255)',
 'ian_2022': 'FLOAT',
 'ida_2022': 'FLOAT',
 'ieg_2022': 'FLOAT',
 'iaa_2022': 'FLOAT',
 'ips_2022': 'FLOAT',
 'ipp_2022': 'FLOAT',
 'ipv_2022': 'FLOAT',
 'matematica_2022': 'FLOAT',
 'portugues_2022': 'FLOAT',
 'ingles_2022': 'FLOAT',
 'instituicao_2022': 'VARCHAR(255)',
 'fase_2023': 'VARCHAR(255)',
 'ian_2023': 'FLOAT',
 'ida_2023': 'FLOAT',
 'ieg_2023': 'FLOAT',
 'iaa_2023': 'FLOAT',
 'ips_2023': 'FLOAT',
 'ipp_2023': 'FLOAT',
 'ipv_2023': 'FLOAT',
 'matematica_2023': 'FLOAT',
 'portugues_2023': 'FLOAT',
 'ingles_2023': 'FLOAT',
 'instituicao_2023': 'VARCHAR(255)',
 'fase_2024': 'VARCHAR(255)',
 'ian_2024': 'FLOAT',
 'ida_2024': 'FLOAT',
 'ieg_2024': 'FLOAT',
 'iaa_2024': 'FLOAT',
 'ips_2024': 'FLOAT',
 'ipp_2024': 'FLOAT',
 'ipv_2024': 'FLOAT',
 'matematica_2024': 'FLOAT',
 'portugues_2024': 'FLOAT',
 'ingles_2024': 'FLOAT',
 'instituicao_2024': 'VARCHAR(255)'}

In [ ]:
sql_columns = ''
i = 0

for col in schema_types:
    col_type = schema_types[col]
    if i > 0:
        sql_columns += ', '
    sql_columns += f'{col} {col_type}'
    i += 1

sql_columns

'genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255)'

In [ ]:
sql_create_table = f'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos({sql_columns})'
sql_create_table

'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos(genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255))'

In [ ]:
try:
    conn = connect_to_db()
    cursor = conn.cursor()

    # 4. Execute the CREATE TABLE statement
    cursor.execute(sql_create_table)

    # 5. Commit the transaction
    conn.commit()
    print("Table created successfully!")
except Exception as e:
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    # 6. Close the cursor and connection
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Failed to create schema | connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



## Subindo dados

In [ ]:
def ingest_data(file, table = 'datathon.passos_magicos'):
    conn = connect_to_db()
    cursor = conn.cursor()
    try:
        with open(file, 'r') as file:
            next(file) # skips header
            cursor.copy_expert(f'COPY {table} FROM STDIN WITH (FORMAT CSV)', file)
    except Exception as e:
        conn.rollback()
        print(f'Failed to insert | {e}')
    finally:
        conn.commit()
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()

---

# Perguntas norteadoras

## 1. Adequação do nível (IAN): Qual é o perfil geral de defasagem dos alunos (IAN) e como ele evolui ao longo do ano?
(Ex.: quantos alunos estão moderadamente ou severamente defasados?)

## 2. Desempenho acadêmico (IDA): O desempenho acadêmico médio (IDA) está melhorando, estagnado ou caindo ao longo das fases e anos?

## 3. Engajamento nas atividades (IEG): O grau de engajamento dos alunos (IEG) tem relação direta com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)?

## 4. Autoavaliação (IAA): As percepções dos alunos sobre si mesmos (IAA) são coerentes com seu desempenho real (IDA) e engajamento (IEG)?

## 5. Aspectos psicossociais (IPS): Há padrões psicossociais (IPS) que antecedem quedas de desempenho acadêmico ou de engajamento?

## 6. Aspectos psicopedagógicos (IPP): As avaliações psicopedagógicas (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?

## 7. Ponto de virada (IPV): Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?

## 8. Multidimensionalidade dos indicadores: Quais combinações de indicadores (IDA + IEG + IPS + IPP) melhor explicam o desempenho global do aluno (INDE)?

## 9. Previsão de risco com Machine Learning: Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem? Construa um modelo preditivo que mostre uma probabilidade do aluno ou aluna entrar em risco de defasagem.

## 10. Efetividade do programa: Os indicadores mostram melhora consistente ao longo do ciclo nas diferentes fases (Quartzo, Ágata, Ametista e Topázio), confirmando o impacto real do programa?

## 11. Insights e criatividade: Você pode adicionar mais insights e pontos de vista não abordados nas perguntas, utilize a criatividade e a análise dos dados para trazer sugestões para a Passos Mágicos.

## Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem?